# Useful decorators

## classmethod / staticmethod

**@classmethod** заставляет декорируемую функцию оперировать самим классом, а не конкретным его инстансом (не требует создания объекта-инстанса класса) 

**@staticmethod** "отвязывает" декорируемую функцию от ее класса в том смысле, что функция теряет доступ к полям класса и полям инстанса класса. Но зато и не требует создания инстанса для своей работы

In [29]:
class MethodsDemo:
    """
    A sample class for classmethods and staticmethods
    """
    NAME = 'methods_demo'
    
    def __init__(self, nickname='default', *args):
        self.nickname = nickname
        
    def do(self, *args):
#         print('self is', self)
#         print(args)
        return args
    
    def access_fields(self, *args):
        return self.NAME, self.nickname
    
    @classmethod
    def do_classmethod(*args):
        return args
    
    @classmethod
    def access_fields_cls(cls, *args):
        return cls.NAME
    
    @classmethod
    def generate_cls(cls, *args):
        return cls(*args)
    
    @staticmethod
    def do_staticmethod(*args):
        return args

In [30]:
dir(MethodsDemo)

['NAME',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'access_fields',
 'access_fields_cls',
 'do',
 'do_classmethod',
 'do_staticmethod',
 'generate_cls']

### Вызовы

In [4]:
MethodsDemo.do_classmethod('param')

(__main__.MethodsDemo, 'param')

In [5]:
MethodsDemo.do_staticmethod('param')

('param',)

In [13]:
MethodsDemo().do('param')

('param',)

### Доступы к полям

In [15]:
MethodsDemo.access_fields()

TypeError: access_fields() missing 1 required positional argument: 'self'

In [19]:
MethodsDemo().access_fields()

('methods_demo', 'default')

In [23]:
MethodsDemo.access_fields_cls()

'methods_demo'

staticmethod не имеет доступов к полям, classmethod имеет доступ только к полям класса

In [26]:
MethodsDemo.generate_cls('new_nick').access_fields()

('methods_demo', 'new_nick')

In [33]:
MethodsDemo.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': '\n    A sample class for classmethods and staticmethods\n    ',
              'NAME': 'methods_demo',
              '__init__': <function __main__.MethodsDemo.__init__(self, nickname='default', *args)>,
              'do': <function __main__.MethodsDemo.do(self, *args)>,
              'access_fields': <function __main__.MethodsDemo.access_fields(self, *args)>,
              'do_classmethod': <classmethod at 0x7f7e50533510>,
              'access_fields_cls': <classmethod at 0x7f7e505335d0>,
              'generate_cls': <classmethod at 0x7f7e50533690>,
              'do_staticmethod': <staticmethod at 0x7f7e505336d0>,
              '__dict__': <attribute '__dict__' of 'MethodsDemo' objects>,
              '__weakref__': <attribute '__weakref__' of 'MethodsDemo' objects>})

In [34]:
MethodsDemo().__dict__

{'nickname': 'default'}

## property

In [41]:
class PropertyShow:
    def __init__(self, var):
        self._var = var
    
    @property
    def x(self):
        return self._var

In [42]:
cls = PropertyShow(42)

In [47]:
cls.x

42

In [45]:
cls._var

42

In [49]:
class NotPropertyShow:
    def __init__(self, var):
        self._var = var
    
    def x(self):
        return self._var

In [50]:
cls = NotPropertyShow(42)

In [51]:
cls.x()

42

In [53]:
cls._var

42

To understand what's going on, read great SO answer [here](https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work-in-python)

## class as a decorator

Экземпляры класса могут быть callable. А если функция может быть декоратором, то что мешает тогда классу? Должен быть определен magic метод `__call__`

In [54]:
class ClassDecorator:
    def __init__(self, func):
        self.entry_str = '[decorating inner func]'
        self.function = func
        self.close_str = '[letting the song flow]'

     
    def __call__(self):
        print(self.entry_str)
        self.function()
        print(self.close_str)

 
 
@ClassDecorator
def function():
    print('Vanderlyle crybaby cry...')

In [56]:
function()

[decorating inner func]
Vanderlyle crybaby cry...
[letting the song flow]


Как и в декораторах-функциях, мы тоже можем возвращать значения (а не только использовать print) и делать параметры

Сделаем таймер

In [60]:
import time

class Timer:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args):
        t_start = time.time()
        res = self.func(*args)
        total_time = time.time() - t_start
        print(f'{self.func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
        return res

In [61]:
@Timer
def slow_func(seconds_to_sleep):
    time.sleep(seconds_to_sleep)
    return 73

In [62]:
slow_func(3)

slow_func((3,)) -> 73 executed in 3.00s


73

## class as decorated

Класс может быть тоже декорирован функцией или классом

In [70]:
def decorator_function(target):

    def decorator_init(self, *args):
        print("Decorator running")
    
    print('I changed __init__ in class')
    target.__init__ = decorator_init

    return target


class Target:

    def __init__(self, *args):
        self.arg_list = args
        print("Target running")


t = Target('one', 1)
t.__dict__

Target running


{'arg_list': ('one', 1)}

In [71]:
@decorator_function
class Target:

    def __init__(self, *args):
        self.arg_list = args
        print("Target running")

I changed __init__ in class


In [72]:
dt = Target('one', 1)
dt.__dict__

Decorator running


{}

# Descriptors

Короткое описание в [доке](https://docs.python.org/3/reference/datamodel.html#invoking-descriptors)

Мы самом деле, мы с вами уже выше познакомились с частным примером такой сущности.

Дескриптор -- это объект с определенными методами `__get__()`, `__set__()` и `__delete__()`

Обычно когда в Python мы хотим получить доступ к атрибуту класса, то сначала мы ищем его в `__dict__` инстанса класса, потом в классе, и далее выше по цепочке наследования.

## Basics

In [74]:
class BestConstant:
    def __get__(self, obj, objtype=None):
        print('Using descriptor')
        return 73

class Number:
    x = 10
    y = BestConstant()  # наш дескриптор

In [79]:
num = Number()

In [80]:
num.x, num.y

Using descriptor


(10, 73)

In [81]:
num.__dict__, Number.__dict__

({},
 mappingproxy({'__module__': '__main__',
               'x': 10,
               'y': <__main__.BestConstant at 0x7f7e2f2de190>,
               '__dict__': <attribute '__dict__' of 'Number' objects>,
               '__weakref__': <attribute '__weakref__' of 'Number' objects>,
               '__doc__': None}))

In [82]:
num.__dict__['y'] = 42

In [83]:
num.y

42

In [84]:
num.__dict__

{'y': 42}

Видим, что мы переписали значение. Дескриптор, у которого определен только get, называется non-data дескриптором

In [85]:
class BestConstantComplete:
    def __get__(self, obj, objtype=None):
        print('using descriptor')
        return obj._y
    
    def __set__(self, obj, value):
        obj._y = value
        
    def __delete__(self, obj):
        del obj._y

class Number:
    y = BestConstantComplete()  # наш дескриптор
    
    def __init__(self, x=10, y=42):
        self.x = x
        self.y = y

In [86]:
num = Number()

In [87]:
num.__dict__

{'x': 10, '_y': 42}

In [89]:
num.y

using descriptor


42

In [90]:
num.y = 73

In [92]:
num.y

using descriptor


73

In [93]:
num.__dict__

{'x': 10, '_y': 73}

In [94]:
num.__dict__['y'] = 100

In [95]:
num.__dict__

{'x': 10, '_y': 73, 'y': 100}

In [97]:
num.y

using descriptor


73

Если у дескриптора определен `__set__`, то Python при попытке достать атрибут по названию, будет доставать сначала дескриптор, даже если в `__dict__` объекта лежит что-то одноименное 

## Closer to Real world

Например, с помощью дескрипторов можно написать логгер запросов к полю класса.

Пример из документации:

In [98]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info(f'Accessing "age" giving {value}')
        return value

    def __set__(self, obj, value):
        logging.info(f'Updating age to {value}')
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1

In [99]:
denis = Person('Denis Belyakov', 23)

INFO:root:Updating age to 23


In [100]:
vars(denis)

{'name': 'Denis Belyakov', '_age': 23}

In [101]:
denis.__dict__

{'name': 'Denis Belyakov', '_age': 23}

In [102]:
denis.age

INFO:root:Accessing "age" giving 23


23

In [103]:
denis.name

'Denis Belyakov'

In [105]:
denis._age

23

## And yes, property

In [106]:
property.__dict__

mappingproxy({'__getattribute__': <slot wrapper '__getattribute__' of 'property' objects>,
              '__get__': <slot wrapper '__get__' of 'property' objects>,
              '__set__': <slot wrapper '__set__' of 'property' objects>,
              '__delete__': <slot wrapper '__delete__' of 'property' objects>,
              '__init__': <slot wrapper '__init__' of 'property' objects>,
              '__new__': <function property.__new__(*args, **kwargs)>,
              'getter': <method 'getter' of 'property' objects>,
              'setter': <method 'setter' of 'property' objects>,
              'deleter': <method 'deleter' of 'property' objects>,
              'fget': <member 'fget' of 'property' objects>,
              'fset': <member 'fset' of 'property' objects>,
              'fdel': <member 'fdel' of 'property' objects>,
              '__doc__': <member '__doc__' of 'property' objects>,
              '__isabstractmethod__': <attribute '__isabstractmethod__' of 'property' objec

    The main motivation for descriptors is to provide a hook allowing objects stored in class variables to control what happens during attribute lookup.

Хорошая [лекция](http://uneex.org/LecturesCMC/PythonIntro2020/11_MiscOOP) на тему

Развернутый [док](https://docs.python.org/3/howto/descriptor.html)

# Inheritance

## Basics

Синтаксис простой

In [107]:
class BetterList(list):
    # поля и методы
    pass

Порядок видимости снизу вверх: объект -> класс -> родительский класс

Для доступа к полям родителя используется `super()`

In [108]:
class Parent:
    def letter(self):
        return 'D'
    
class Child(Parent):
    def letter(self):
        return super().letter() + 'B'

In [109]:
a = Parent()
b = Child()

In [110]:
a.letter()

'D'

In [111]:
b.letter()

'DB'

In [112]:
class Parent:
    def __init__(self, letter='D'):
        self._letter = letter
        
    def get_letter(self):
        return self._letter
    
class Child(Parent):
    def __init__(self, parent_letter='D', child_letter='B'):
        super().__init__(parent_letter)
        print(f'parent inited {self._letter}')
        self.new_letter = child_letter
        
    def get_child_letter(self):
        return self.new_letter

In [113]:
a = Child()

parent inited D


In [114]:
a.get_letter()

'D'

In [115]:
a.get_child_letter()

'B'

In [116]:
a.__dict__

{'_letter': 'D', 'new_letter': 'B'}

## Protecting names

Фичи наследования в доопределении пространства имен и в возможности их перегрузки  

Предполагается, что если пользователь переопредляет имя, значит он этого хочет 

Если создатель класса не хочет, чтобы какое-то поле было явно перегружено, то можно, например, использовать дескрипторы или особый нейминг

Есть общее соглашение, что переменные, названные с `_` не надо трогать вне класса, в котором они объявлены

Также в Python предусмотрен механизм с названием, начинающися с `__`

In [125]:
class StrictParent:
    def __init__(self):
        self.__letter = 'D'

class Child(StrictParent):
    def __init__(self):
        super().__init__()
        self._letter = 'B'

In [126]:
a = Child()

In [127]:
a.__dict__

{'_StrictParent__letter': 'D', '_letter': 'B'}

## Тонкости

Вспомним игрушечный пример

In [2]:
class Parent:
    def __init__(self, letter='D'):
        self._letter = letter
        
    def get_letter(self):
        return self._letter
    
class Child(Parent):
    def __init__(self, parent_letter='D', child_letter='B'):
        super().__init__(parent_letter)
        print(f'parent inited {self._letter}')
        self.new_letter = child_letter
        
    def get_child_letter(self):
        return self.new_letter

### Предположим, вы хотите отнаследоваться от стандартного типа. Что может пойти не так

In [3]:
class DoubleTroubleDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [5]:
dd = DoubleTroubleDict(one=1)
dd

{'one': 1}

In [7]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [9]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

У нас нет никакой гарантии, что `__setitem__` наследника вызывается при обновлении словаря!

In [21]:
class TroubleAnswerDict(dict):
    def __getitem__(self, key):
        return 42
    
bd = TroubleAnswerDict(a='answer')
bd['a']

42

In [22]:
ordinary_d = {}
ordinary_d.update(bd)
ordinary_d

{'a': 'answer'}

Наш `__getitem__` проигнорировали..

### Как надо?

In [12]:
from collections import UserDict

In [17]:
class DoubleDict(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [18]:
dd = DoubleDict(one=1)
dd

{'one': [1, 1]}

In [19]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [20]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [28]:
class AnswerDict(UserDict):
    def __getitem__(self, key):
        return 42
    
bd = AnswerDict(a='answer')
bd['a']

42

In [30]:
ordinary_d = {}
ordinary_d.update(bd)
ordinary_d

{'a': 42}

## ABC

`Types are defined by their supported operations`

Для того, чтобы что-то работало как последовательность, не обязательно определять много методов. Достаточно объявить `__getitem__`

In [32]:
class Vovels:
    def __getitem__(self, i):
        return 'AIEUO'[i]

vovels = Vovels()
vovels[0], vovels[-1]

('A', 'O')

In [33]:
for v in vovels: print(v)

A
I
E
U
O


Однако несмотря на простоту, мы не определили len! Казалось бы, один из базовых методов, но можно и без него

In [35]:
len(vovels)

TypeError: object of type 'Vovels' has no len()

Можно сказать, что в данном случае мы реализовали часть *динамического протокола* последовательности, и при этом все равно смогли использовать ее функционал. Однако, в таком случае мы можем столкнуться с проблемами, когда передадим нашу "последовательность" куда-то, где от нее будут ожидать реализацию хотя бы метода длины

Это отличный пример **duck typing** -- игнорирование конкретного типа объекта, если он определяет конкретные необходимые методы

Нам не обязательно создавать наследника класса последовательности, чтобы использовать объект так, как мы хотим. Python делает много работы за нас в этом случае

Но что если мы явно хотим сделать некоторые методы обязательными? Здесь нам на помощь приходят **abstract base classes**
https://docs.python.org/3/library/abc.html

### **Goose typing**:
- Наследуемся от ABC чтобы явно продемонстрировать следование объявленному интерфейсу
- Проверка типа isinstance и issubclass осуществляется для интерфейса

In [40]:
from collections import namedtuple, abc

class SequenceVovels(abc.Sequence):
    def __getitem__(self, i):
        return 'AIEUO'[i]

In [41]:
vov = SequenceVovels()

TypeError: Can't instantiate abstract class SequenceVovels with abstract methods __len__

In [42]:
class SequenceVovels(abc.Sequence):
    
    VOVELS = 'AIEUO'
    
    def __getitem__(self, i):
        return VOVELS[i]
    
    def __len__(self):
        return len(self.VOVELS)

In [43]:
vov = SequenceVovels()

### Можно объявить свой ABC

In [47]:
import abc
class MyABC(abc.ABC):
    
    @abc.abstractmethod
    def do_smth(self, a):
        """does something"""
    
    @staticmethod
    @abc.abstractmethod  # should be inner, order matters
    def do_smth_static(a):
        """a staticmethod that does something"""
        
    @classmethod
    @abc.abstractmethod  # should be inner, order matters
    def do_smth_class(cls, a):
        """a classmethod that does something"""

### Варианты проверки типов

Предположим, у нас есть некоторый объект `maybe_complex` и мы хотим проверить, можно ли использовать его как Complex number

**1 способ**

```python
from typing import SupportsComplex

if isinstance(maybe_comlex, (complex, SupportsComplex)):
    # do smth that requires our object to be complex
    else:
        raise TypeError('maybe_complex should be convertible to complex')
```

**2 способ**

```python

import numbers

if isinstance(maybe_comlex, (numbers.Complex)):
    # do smth that requires our object to be complex
    else:
        raise TypeError('maybe_complex should be an instance of complex')
```

**3 способ aka Fail Fast or EAFP** *‘Easier to Ask for Forgiveness than Permission’*



```python

try:
    c = complex(maybe_complex)
except TypeError as e:
    raise TypeError('maybe_complex should be convertible to complex')
```

Несколько советов про наследование:

1. Не переиспользуйте наследование for fun там, где этого можно избежать
2. Понимайте, почему в конкретном месте используется наследование
3. Если вам важно соблюдение интерфейса, обозначьте это явно с помощью ABC
4. Будьте осторожны с наследованием от стандартных типов. Используйте модуль Collections

# Exceptions

Оператор **try-except-else-finally**

In [131]:
a = '123'

try:
    a[0] = '10'
except BaseException:
    print('impossible to modify object')

impossible to modify object


In [129]:
a[0] = '10'

TypeError: 'str' object does not support item assignment

In [132]:
TypeError.mro()

[TypeError, Exception, BaseException, object]

In [141]:
a = [1, 2, 3]
a = '123'

try:
    a[0] = '10'
except Exception as e:
    print('impossible to modify object')
else:
    print('possible to modify object', a)

impossible to modify object


Исключения — не «ошибки», а способ обработки некоторых условий не там, где они были обнаружены.

In [135]:
raise TypeError

TypeError: 

In [136]:
raise TypeError('message')

TypeError: message

Хорошая [лекция](http://uneex.org/LecturesCMC/PythonIntro2020/10_Inheritance) по механизму наследования в т.ч. с примерами исключений 

---

# *Bonus:*

Напишите декоратор-класс, который бы:
1. Проверял входные аргументы и вывод функции на соответствие типам, указанным в аннотациях 
2. Проверял, что все численные типы находятся в диапазоне от -1000 до 1000
3. Длины типов-последовательностей не превосходят 1000
4. В случае ошибок валидации возвращал информативное сообщение об ошибке в виде Exception
5. Мог бы быть отключаемым по параметрам

Для решения используйте дескрипторы, вам очень помогут упомянутые сегодня ссылки 

In [142]:
def func(a: int, b: str) -> str:
    pass

---

https://docs.python.org/3/howto/descriptor.html#validator-class

In [1]:
import collections.abc

from abc import ABC, abstractmethod
from functools import wraps
from typing import Iterable


class Validator(ABC):

    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass
    

class NumberValidator(Validator):

    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue is not None and value < self.minvalue:
            raise ValueError(
                f'Expected {value!r} to be at least {self.minvalue!r}'
            )
        if self.maxvalue is not None and value > self.maxvalue:
            raise ValueError(
                f'Expected {value!r} to be no more than {self.maxvalue!r}'
            )

class SeqValidator(Validator):
    def __init__(self, minlen=None, maxlen=None):
        self.minlen = minlen
        self.maxlen = maxlen

    def validate(self, iterable):
        if not isinstance(iterable, collections.abc.Iterable):
            raise TypeError(f'Expected {value!r} to be iterable')
        
        if self.minlen is not None and len(iterable) < self.minlen:
            raise ValueError(
                f'Expected lenght of sequence = {len(iterable)} to be at least len {self.minlen}'
            )
        if self.maxlen is not None and len(iterable) > self.maxlen:
            raise ValueError(
                f'Expected lenght of sequence = {len(iterable)} to be no more than {self.maxlen}'
            )

In [2]:
class TypeChecker:
    
    def __init__(self, args_to_check: Iterable[str]):
        self.args_to_check = set(args_to_check)
        
        self.number_value_checker = NumberValidator(minvalue=-1000, maxvalue=1000)
        self.seq_len_checker = SeqValidator(maxlen=1000)

    def __call__(self, inner_function: callable):
        self.inner_func = inner_function

        @wraps(inner_function)
        def _check(*args):
            self.res = self.inner_func(*args)
            annotations = self.inner_func.__annotations__
            for annot_arg, real_arg in zip(annotations, args):
                if annot_arg in self.args_to_check:
                    if not isinstance(real_arg, annotations[annot_arg]):
                        raise TypeError(f'Argument {annot_arg} is {type(real_arg)} instead of {annotations[annot_arg]}')
                    
                    try:
                        if isinstance(real_arg, (int, float)):
                            self.number_value_checker.validate(real_arg)
                        elif isinstance(real_arg, collections.abc.Iterable):
                            self.seq_len_checker.validate(real_arg)
                    except ValueError as e:
                        raise ValueError(f'Argument "{annot_arg}" has wrong value!', e)
            
            if 'return' in annotations and not isinstance(self.res, annotations['return']):
                raise TypeError(f'Return type is {type(self.res)} instead of {annotations["return"]}')

            
        return _check


In [475]:
@TypeChecker(['a', 'b', 'return'])
def fun(a: int, b: str, c: tuple, d: int) -> str:
    return 42

In [476]:
fun(1000, '1', tuple(list(range(10000))), 10)

TypeError: Return type is <class 'int'> instead of <class 'str'>

In [477]:
fun('10', '1', tuple(list(range(10000))), 10)

TypeError: Argument a is <class 'str'> instead of <class 'int'>

In [478]:
fun(2000, '1', tuple(list(range(10000))), 10)

ValueError: ('Argument "a" has wrong value!', ValueError('Expected 2000 to be no more than 1000'))

In [479]:
fun(500, '1'*2000, (1, 2), 10)

ValueError: ('Argument "b" has wrong value!', ValueError('Expected lenght of sequence = 2000 to be no more than 1000'))